In [1]:
import pandas as pd
import numpy as np
import re
import os
from ast import literal_eval

In [2]:
from google.colab import drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/DataMan_Songs/Dataset/'

Mounted at /content/drive


In [3]:
df = pd.read_csv(dataset_path + '1_Dataset_Songs_Features.csv', index_col = 'Unnamed: 0')
## Read Technque rows as list
df.Technique = df.Technique.apply(literal_eval)
df.tail(50)
## There are many errors splitting 'PLayer' and 'Technique'.
## Many rows have cyrillic characters
## Some rows have invalid Player or/and Band or/and Song.
## Some rows don't have Difficulty level

,Band,Song,Player,Technique,Difficulty,Score
7236,Gilberto Rojas Enríquez,Ojos Azules,voz (Staff 2),[Violin],NaN,NaN
7237,Cesar Miró,Todos Vuelven,voz2 (Staff 2),[Orchestral Harp],NaN,NaN
7238,Chabuca Granda,La flor de la Canela,Another Bass(plain),[Pizzicato Strings],NaN,NaN
7239,Satoru Kōsaki,Dokuzetsu,jank bass,[Acoustic Grand Piano],NaN,NaN
7240,Tesuto,sd,Track 4,[Acoustic Guitar (steel)],Advanced,8/8
7241,Dave Matthews,So Much To Say,Baritone Sax,[],NaN,NaN
7242,Gfsgf,fgfdgfgd,Cliff Burton (Distortion),[Distortion Guitar],Intermediate,3/8
7243,Fdfsdfdsf,didid,Cliff Burton (Distortion),[Distortion Guitar],Intermediate,3/8
7244,Grffdgf,fggfdgf,Cliff Burton (Distortion),[Distortion Guitar],Beginner,1/8
7245,Bfggbff,fgdfgfdgd,Cliff Burton (Distortion),[Distortion Guitar],Beginner,1/8


Some analysis:
- Check duplicates
- Check repetitions in each columns
- Check Difficulty and Score columns

In [4]:
## Build a Dataframe contaings all erros (for plot)
err_df = pd.DataFrame()

In [5]:
## Shape df
print('Original Dataset: {} rows | {} columns'.format(len(df), len(df.columns)))
print(' ')
err_df['Original Shape'] = df.shape[0]


## Check duplicated
dup = df.loc[df.duplicated(subset=['Band', 'Song', 'Player']) == True]
print('N° duplicates: {}'.format(len(dup)))
err_df['Duplicates'] = [len(dup)]
df.drop_duplicates(subset=['Band', 'Song', 'Player'], inplace = True)
print('No duplicates Dataset: {} rows | {} columns'.format(len(df), len(df.columns)))


Original Dataset: 7286 rows | 6 columns
 
N° duplicates: 78
No duplicates Dataset: 7208 rows | 6 columns


In [6]:
err_df['Original Shape'] = 7286

In [7]:
## Control if there are repetition in consecutive columns
## Define function: object will be every rows

def repetition_col(x):
    
    columns=  df.columns
    band = [x[columns[0]]] 
    song = [x[columns[1]]]
    player = [x[columns[2]]]
    technique = x[columns[3]]
    
    errors = 0
    ok = 0
    
    if band in song or song in player or player in technique:
        errors += 1
        print('Repetition')
    else:
        ok += 1
    
    return(errors, ok)

## Check if there are repetitions through columns

In [8]:
dt = pd.DataFrame(df.apply(lambda x: repetition_col(x), axis = 1).to_list(), columns = ['Errors', 'Ok'])
print('Len dataset: {}'.format(len(df)))
print('N° right divisions: ', dt.groupby('Errors').sum().values[0][0])
print('N° wrong divisions: ', dt.groupby('Ok').sum().values[0][0])
## There aren't repetitions

Len dataset: 7208
N° right divisions:  7208
N° wrong divisions:  0


## Check and drop rows with not defined Score or Difficulty

In [9]:
i2 = df[df['Difficulty'].isnull()].index
err_df['NanScore'] = len(i2)
print(len(i2))
df.drop(i2, inplace=True)
# df.drop('index', inplace=True, axis = 1)
df.reset_index(inplace=True)
print(len(df))
df

2110
5098


,index,Band,Song,Player,Technique,Difficulty,Score
0,0,The Beatles,Blackbird,Paul McCartney,"[Guitar, Acoustic Guitar (nylon)]",Intermediate,3/8
1,1,Isaac Albéniz,Asturias (Leyenda),Ýz 1,[Acoustic Guitar (nylon)],Intermediate,5/8
2,2,AC/DC,Back In Black,Angus Young (Lead),[Distortion Guitar],Intermediate,3/8
3,3,Pachelbel,Canon in D Major,NaN,[Track: Acoustic Guitar (nylon)],Intermediate,4/8
4,4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,[Acoustic Guitar (steel)],Beginner,2/8
...,...,...,...,...,...,...,...
5093,7277,Проэкт,Музыка,Distorted Bass,[Distortion Guitar],Intermediate,4/8
5094,7280,Paul McCartney,Uncle Albert/Admiral Halsey,Contrabass,[Electric Guitar (jazz)],Beginner,1/8
5095,7283,My Bloody Valentine,Cigarette in Your Bed,Fuzz Bass,[Distortion Guitar],Beginner,1/8
5096,7284,Tadzik,Красивая музыка,Bass,[Overdriven Guitar],Intermediate,3/8


## Check if there are cyrillic characters and false name

In [10]:
# ## Check not english caracter
not_english = []
def eng_character(x):
    ## Ex: Blink 182
    eng = re.compile('[a-zA-Z0-9]')
       
    if eng.findall(x):
        print('English: {}'.format(x))
    else:
        print('Not english: {}'.format(x))
        not_english.append(x)
#         not_english_index.append(x.index)
        
    return()


df['Song'].apply(lambda x: eng_character(x))
df['Band'].apply(lambda x: eng_character(x))

err_df['NotEnglishCharacters'] = len(not_english)

## Plot

Output streaming troncato alle ultime 5000 righe.
English: Lamb Of God
English: Stevie Ray Vaughan
English: Megadeth
English: Death
English: Kana-Boon
English: Jeff Buckley
English: Skillet
English: Extreme
English: Lil Peep
English: Seether
English: Van Halen
English: Aerosmith
English: Megadeth
English: Steve Vai
English: Tool
English: Eric Clapton
English: Jason Becker
English: Lamb Of God
English: Duman
English: Iron Maiden
English: Death
English: Cream
English: Lamb Of God
English: Mountain
English: Simon And Garfunkel
English: Ozzy Osbourne
English: Metallica
English: Iron Maiden
English: Metallica
English: Limp Bizkit
English: Paul Gilbert
English: Gary Moore
English: Francisco Tarrega
English: Spanish Song
English: Metallica
English: Sublime
English: Opeth
English: Led Zeppelin
English: Jimi Hendrix
English: Pearl Jam
English: Dream Theater
English: Radiohead
English: Tenacious D
English: System of a Down
English: Stevie Ray Vaughan
English: Judas Priest
English: Metallica
Engl

In [11]:
## Drop cyrillic rows
print(len(df))
for el in not_english:
    i = df[df['Song'] == el].index
    l = df[df['Band'] == el].index

    df.drop(i, inplace = True)
    try:
        df.drop(l, inplace = True)
    except:
        continue

print(len(df))


5098
4822


In [12]:
## Find rows with spelling mistakes or false name (es:Bfggbff)
# ## Check not english caracter
not_english2 = []
def eng_character(x):

    not_eng2 = re.compile('[b-df-hj-np-tv-xz]{5}')
       
    if not_eng2.findall(x):
        print('No Sense: {}'.format(x))
        not_english2.append(x)

        
    return()


df['Song'].apply(lambda x: eng_character(x))
df['Band'].apply(lambda x: eng_character(x))

No Sense: Deutschland
No Sense: Sdfsdf
No Sense: dozhdmz_dekabrya
No Sense: Nightstalker
No Sense: lbjkdfbjkdfbsjk
No Sense: Gpcphcpu
No Sense: fcfbdgvc
No Sense: lshidfgfsdo
No Sense: Metallllll Songggggg
No Sense: rdrdrdrd
No Sense: asredrefffffffffffffffffffffffffyaaaaaaaatombo
No Sense: NUMetaLmgmgm
No Sense: Deutschland
No Sense: hytgfrcdexsz
No Sense: }asdwqewqewq´{xzzxvxz{fwq{/3"!#!"
No Sense: ´´p{´p´p{p´{p´p´{pjhknhgjtyrt
No Sense: Hdsfgdfdh644652
No Sense: tlsbr
No Sense: sdihhfsdu
No Sense: zxcvbn
No Sense: zxcvb
No Sense: asdfghj
No Sense: asdfghj
No Sense: Asdgkjhna
No Sense: gfjgj
No Sense: gjkjdbyf gtcyb [vrblc
No Sense: 6365gbnjm46bi
No Sense: ,nm,bnm,bnmbnm,m
No Sense: Bucket of Afterbirth (by jnsrkr)
No Sense: 6365gbnjm46bi
No Sense: crxdfveg
No Sense: ni idea jsjsjs
No Sense: hhhhhhh
No Sense: tththtthth
No Sense: rtrtr
No Sense: warrrrrsss
No Sense: ddassssssss
No Sense: Svetstsdasd
No Sense: gi3norergojkngrejno
No Sense: sdffsdsdf
No Sense: tththtthth
No Sense: fajb

0       ()
1       ()
2       ()
3       ()
4       ()
        ..
5088    ()
5089    ()
5091    ()
5094    ()
5095    ()
Name: Band, Length: 4822, dtype: object

In [13]:
for el in not_english2:
    if el == 'Deutschland':
        print('Deutschland')
    
not_english2.remove('Deutschland')

err_df['NoSenseCharacters'] = len(not_english2)


Deutschland
Deutschland


In [14]:
## Drop no sense rows
print(len(df))
for el in not_english2:
    i = df[df['Song'] == el].index
    l = df[df['Band'] == el].index

    df.drop(i, inplace = True)
    try:
        df.drop(l, inplace = True)
    except:
        continue

print(len(df))

4822
4743


Check how many rows don't have player or take mistakes 

In [15]:
## Make uniform styles
## If player keep 1 instruments he's not formatted well or it's an error
## Divide for instruments
df.reset_index(inplace=True)
basic_instruments = ['Guitar', 'Bass', 'Drums', 'Piano']
re_basic_instruments = [re.compile('({})'.format(instrument[1:])) for instrument in basic_instruments]
guitar = []
bass = [] 
drums = [] 
piano = []

def instrumental_division(x):
    
    if re_basic_instruments[0].findall(x):
        guitar.append(x)
    if re_basic_instruments[1].findall(x):
        bass.append(x)
    if re_basic_instruments[2].findall(x):
        drums.append(x)
    if re_basic_instruments[3].findall(x):
        piano.append(x)
       
    return(guitar, bass, drums, piano)
    
    

df['Player'].astype(str).apply(lambda x: instrumental_division(x))  


0       ([Guitar 1, Serenata para Guitarra, Nylon Guit...
1       ([Guitar 1, Serenata para Guitarra, Nylon Guit...
2       ([Guitar 1, Serenata para Guitarra, Nylon Guit...
3       ([Guitar 1, Serenata para Guitarra, Nylon Guit...
4       ([Guitar 1, Serenata para Guitarra, Nylon Guit...
                              ...                        
4738    ([Guitar 1, Serenata para Guitarra, Nylon Guit...
4739    ([Guitar 1, Serenata para Guitarra, Nylon Guit...
4740    ([Guitar 1, Serenata para Guitarra, Nylon Guit...
4741    ([Guitar 1, Serenata para Guitarra, Nylon Guit...
4742    ([Guitar 1, Serenata para Guitarra, Nylon Guit...
Name: Player, Length: 4743, dtype: object

In [16]:
err_df['GuitarErrors'] = len(guitar) 
err_df['BassErrors'] = len(bass) 
err_df['DrumsErrors'] = len(drums) 
err_df['PianoErrors'] = len(piano) 


print('Guitar mistakes: {}'.format(len(guitar)), 
     'Bass mistakes: {}'.format(len(bass)), 
     'Drums mistakes: {}'.format(len(drums)), 
     'Piano mistakes: {}'.format(len(piano)))

df['Player'] = df['Player'].astype(str).apply(lambda x: x.strip('[').strip(']'))
# df.reset_index(inplace = True)

instruments = drums + guitar + bass + piano
## Find rows with errors
index_err = []
for rows in df['Player'].values:
    if rows in instruments:
        index_err.append(df[df['Player'] == rows])


Guitar mistakes: 1896 Bass mistakes: 332 Drums mistakes: 0 Piano mistakes: 6


In [17]:
## Build a new dataset with all mistakes
db = index_err[0]
for el in index_err[1:]:
    db = pd.concat([db, el], ignore_index = True)

db.drop_duplicates(['Band', 'Song'], inplace= True)
db = db.drop('level_0', axis = 1)
db = db.drop('index', axis = 1)
db.reset_index(inplace=True)
err_df['PlayerErrors'] = len(db)
db.head()

,index,Band,Song,Player,Technique,Difficulty,Score
0,0,Kansas,Dust In The Wind,Guitar 1,[Acoustic Guitar (nylon)],Intermediate,3/8
1,1,Metallica,Nothing Else Matters (Acoustic),Guitar 1,[Acoustic Guitar (steel)],Intermediate,5/8
2,2,Theme Songs,Pirates Of The Caribbean,Guitar 1,[Acoustic Guitar (steel)],Beginner,2/8
3,3,Dick Dale,Misirlou,Guitar 1,[Electric Guitar (clean)],Intermediate,4/8
4,4,Blue Oyster Cult,Don't Fear The Reaper,Guitar 1,[Electric Guitar (clean)],Beginner,2/8


In [18]:
## Move 'Player' to 'Technique'
for x in range(len(db)):    
    if db['Player'][x] not in db['Technique'][x]:
        db['Technique'][x].append(db['Player'][x])

db.drop('Player', axis = 1, inplace = True)
db.drop('index', inplace=True, axis = 1)
db.drop('Difficulty', inplace=True, axis = 1)
db.drop('Score', inplace=True, axis = 1)
db['Player'] = [np.nan for el in range(len(db))]
db

## Merge df and db
df = df.merge(db, on=['Band', 'Song'], how='left')
df.head(3)

,level_0,index,Band,Song,Player_x,Technique_x,Difficulty,Score,Technique_y,Player_y
0,0,0,The Beatles,Blackbird,Paul McCartney,"[Guitar, Acoustic Guitar (nylon)]",Intermediate,3/8,NaN,NaN
1,1,1,Isaac Albéniz,Asturias (Leyenda),Ýz 1,[Acoustic Guitar (nylon)],Intermediate,5/8,NaN,NaN
2,2,2,AC/DC,Back In Black,Angus Young (Lead),[Distortion Guitar],Intermediate,3/8,NaN,NaN


In [19]:
## Add 'Technique_y' last element to Technique --> Add 'Player' instrument into the right column
for x in range(len(df)):
    if type(df['Technique_y'][x]) == list:
        df['Technique_x'][x].append(df['Technique_y'][x][-1])
        df['Player_x'][x] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [20]:
df.drop(['Technique_y', 'Player_y'], axis=1, inplace=True)
df.drop(['level_0', 'index'], axis=1, inplace=True)
df.rename(columns={'Player_x': 'Player', 'Technique_x': 'Technique'}, inplace=True)

## Remove duplicates from technique lists and clean them from any instrument != guitar

In [21]:
## Remove duplicates
df['Technique'] = df['Technique'].apply(lambda x: list(dict.fromkeys(x)))
df

,Band,Song,Player,Technique,Difficulty,Score
0,The Beatles,Blackbird,Paul McCartney,"[Guitar, Acoustic Guitar (nylon)]",Intermediate,3/8
1,Isaac Albéniz,Asturias (Leyenda),Ýz 1,[Acoustic Guitar (nylon)],Intermediate,5/8
2,AC/DC,Back In Black,Angus Young (Lead),[Distortion Guitar],Intermediate,3/8
3,Pachelbel,Canon in D Major,nan,[Track: Acoustic Guitar (nylon)],Intermediate,4/8
4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,[Acoustic Guitar (steel)],Beginner,2/8
...,...,...,...,...,...,...
4738,Your Mumies,play.,Track 1,[Distortion Guitar],Intermediate,3/8
4739,Slaver,The moon,NaN,"[Distortion Guitar, Bass Fuzz]",Intermediate,3/8
4740,Vundabar,Alien Blues,NaN,"[Distortion Guitar, Distortion Bass]",Intermediate,4/8
4741,Paul McCartney,Uncle Albert/Admiral Halsey,NaN,"[Electric Guitar (jazz), Contrabass]",Beginner,1/8


In [22]:
## Keep only guitar type
def find_guitar(x):
    guitar_tech = []
    for el in x:
        try:
            el = el.replace('Track: ', '')
        except:
            continue

        if re_basic_instruments[0].findall(el):
            guitar_tech.append(el)
        
    return(guitar_tech)

df['Technique'] = df['Technique'].apply(lambda x: find_guitar(x))

In [23]:
df['Player'] = df['Player'].astype(str).apply(lambda x: re.sub(r'(Track 1)', 'nan', x))
df['Song'] = df['Song'].astype(str).apply(lambda x: re.sub(r"\((.*?)\)", "", x))
df['Song'] = df['Song'].astype(str).apply(lambda x: re.sub(r" $", "", x))
df

,Band,Song,Player,Technique,Difficulty,Score
0,The Beatles,Blackbird,Paul McCartney,"[Guitar, Acoustic Guitar (nylon)]",Intermediate,3/8
1,Isaac Albéniz,Asturias,Ýz 1,[Acoustic Guitar (nylon)],Intermediate,5/8
2,AC/DC,Back In Black,Angus Young (Lead),[Distortion Guitar],Intermediate,3/8
3,Pachelbel,Canon in D Major,nan,[Acoustic Guitar (nylon)],Intermediate,4/8
4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,[Acoustic Guitar (steel)],Beginner,2/8
...,...,...,...,...,...,...
4738,Your Mumies,play.,nan,[Distortion Guitar],Intermediate,3/8
4739,Slaver,The moon,nan,[Distortion Guitar],Intermediate,3/8
4740,Vundabar,Alien Blues,nan,[Distortion Guitar],Intermediate,4/8
4741,Paul McCartney,Uncle Albert/Admiral Halsey,nan,[Electric Guitar (jazz)],Beginner,1/8


In [24]:
df.to_csv(dataset_path + '1_Dataset_Song_Features_clean.csv', index=False)

In [25]:
# err_df.to_csv(dataset_path + '1_Df_Erros.csv', index=False)